### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [2]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
cme_url = base_url+CME+"?startDate="+startDate+"&endDate="+endDate+"&api_key="+NASA_API_KEY

In [3]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(cme_url)

In [4]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()

In [5]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[0], indent=4))

{
    "activityID": "2013-05-01T03:12:00-CME-001",
    "catalog": "M2M_CATALOG",
    "startTime": "2013-05-01T03:12Z",
    "instruments": [
        {
            "displayName": "SOHO: LASCO/C2"
        },
        {
            "displayName": "SOHO: LASCO/C3"
        },
        {
            "displayName": "STEREO A: SECCHI/COR2"
        },
        {
            "displayName": "STEREO B: SECCHI/COR2"
        }
    ],
    "sourceLocation": "",
    "activeRegionNum": null,
    "note": "",
    "submissionTime": "2013-08-07T16:54Z",
    "versionId": 1,
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
    "cmeAnalyses": [
        {
            "isMostAccurate": true,
            "time21_5": "2013-05-01T07:07Z",
            "latitude": 12.0,
            "longitude": -120.0,
            "halfAngle": 36.0,
            "speed": 860.0,
            "type": "C",
            "featureCode": "null",
            "imageType": null,
            "measurementTechnique": "null",
   

In [6]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.DataFrame(cme_json)
# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[["activityID","startTime","linkedEvents"]]
cme_df

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
...,...,...,...
5519,2024-05-01T06:36:00-CME-001,2024-05-01T06:36Z,None
5520,2024-05-01T11:36:00-CME-001,2024-05-01T11:36Z,None
5521,2024-05-01T12:36:00-CME-001,2024-05-01T12:36Z,None
5522,2024-05-01T17:36:00-CME-001,2024-05-01T17:36Z,None


In [7]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df = cme_df.dropna(subset="linkedEvents").reset_index(drop=True)
cme_df

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
4,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...
...,...,...,...
1018,2024-04-24T01:48:00-CME-001,2024-04-24T01:48Z,[{'activityID': '2024-04-26T00:17:00-IPS-001'}]
1019,2024-04-24T15:05:00-CME-001,2024-04-24T15:05Z,[{'activityID': '2024-04-24T14:02:00-FLR-001'}]
1020,2024-04-25T18:24:00-CME-001,2024-04-25T18:24Z,[{'activityID': '2024-04-25T17:03:00-FLR-001'}]
1021,2024-04-30T00:36:00-CME-001,2024-04-30T00:36Z,[{'activityID': '2024-04-30T00:46:00-FLR-001'}]


In [8]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for entry in cme_df.index:
    # Iterate over each dictionary in the list
    entry_activityID = cme_df.iloc[entry]["activityID"]
    entry_startTime = cme_df.iloc[entry]["startTime"]
    entry_linkedEvents = cme_df.iloc[entry]["linkedEvents"]
    
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
    for item in entry_linkedEvents:
        expanded_rows.append({"activityID":entry_activityID,"startTime":entry_startTime,"linkedEvents":item})
# Create a new DataFrame from the expanded rows
cme_df_expanded = pd.DataFrame(expanded_rows)
cme_df_expanded

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'}
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'}
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'}
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'}
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'}
...,...,...,...
1709,2024-04-24T01:48:00-CME-001,2024-04-24T01:48Z,{'activityID': '2024-04-26T00:17:00-IPS-001'}
1710,2024-04-24T15:05:00-CME-001,2024-04-24T15:05Z,{'activityID': '2024-04-24T14:02:00-FLR-001'}
1711,2024-04-25T18:24:00-CME-001,2024-04-25T18:24Z,{'activityID': '2024-04-25T17:03:00-FLR-001'}
1712,2024-04-30T00:36:00-CME-001,2024-04-30T00:36Z,{'activityID': '2024-04-30T00:46:00-FLR-001'}


In [9]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(input_dict):
        try:
                return input_dict["activityID"]
        # Log the error or print it for debugging
        except (ValueError, TypeError) as e:        
                return e

extract_activityID_from_dict(cme_df_expanded.loc[0,'linkedEvents'])

'2013-05-04T04:52:00-IPS-001'

In [10]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
cme_df_expanded["GST_ActivityID"] = cme_df_expanded["linkedEvents"].apply(lambda x: extract_activityID_from_dict(x))
cme_df_expanded

,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001
...,...,...,...,...
1709,2024-04-24T01:48:00-CME-001,2024-04-24T01:48Z,{'activityID': '2024-04-26T00:17:00-IPS-001'},2024-04-26T00:17:00-IPS-001
1710,2024-04-24T15:05:00-CME-001,2024-04-24T15:05Z,{'activityID': '2024-04-24T14:02:00-FLR-001'},2024-04-24T14:02:00-FLR-001
1711,2024-04-25T18:24:00-CME-001,2024-04-25T18:24Z,{'activityID': '2024-04-25T17:03:00-FLR-001'},2024-04-25T17:03:00-FLR-001
1712,2024-04-30T00:36:00-CME-001,2024-04-30T00:36Z,{'activityID': '2024-04-30T00:46:00-FLR-001'},2024-04-30T00:46:00-FLR-001


In [11]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
cme_df_expanded = cme_df_expanded.dropna(subset="GST_ActivityID").reset_index(drop=True)
cme_df_expanded

,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001
...,...,...,...,...
1709,2024-04-24T01:48:00-CME-001,2024-04-24T01:48Z,{'activityID': '2024-04-26T00:17:00-IPS-001'},2024-04-26T00:17:00-IPS-001
1710,2024-04-24T15:05:00-CME-001,2024-04-24T15:05Z,{'activityID': '2024-04-24T14:02:00-FLR-001'},2024-04-24T14:02:00-FLR-001
1711,2024-04-25T18:24:00-CME-001,2024-04-25T18:24Z,{'activityID': '2024-04-25T17:03:00-FLR-001'},2024-04-25T17:03:00-FLR-001
1712,2024-04-30T00:36:00-CME-001,2024-04-30T00:36Z,{'activityID': '2024-04-30T00:46:00-FLR-001'},2024-04-30T00:46:00-FLR-001


In [12]:
# print out the datatype of each column in this DataFrame:
cme_df_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   activityID      1714 non-null   object
 1   startTime       1714 non-null   object
 2   linkedEvents    1714 non-null   object
 3   GST_ActivityID  1714 non-null   object
dtypes: object(4)
memory usage: 53.7+ KB


In [13]:
# Convert the 'GST_ActivityID' column to string format 
cme_df_expanded["GST_ActivityID"] = cme_df_expanded["GST_ActivityID"].astype("string")
# Convert startTime to datetime format  
cme_df_expanded["startTime"] = pd.to_datetime(cme_df_expanded["startTime"])
# Rename startTime to startTime_CME and activityID to cmeID
cme_df_expanded.rename(columns={"startTime":"startTime_CME","activityID":"cmeID"},inplace=True)
# Drop linkedEvents
cme_df_expanded.drop(columns=["linkedEvents"],inplace=True)
# Verify that all steps were executed correctly
cme_df_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   cmeID           1714 non-null   object             
 1   startTime_CME   1714 non-null   datetime64[ns, UTC]
 2   GST_ActivityID  1714 non-null   string             
dtypes: datetime64[ns, UTC](1), object(1), string(1)
memory usage: 40.3+ KB


In [14]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
cme_df_expanded = cme_df_expanded[cme_df_expanded["GST_ActivityID"].str.contains("GST")].reset_index(drop=True)
cme_df_expanded

,cmeID,startTime_CME,GST_ActivityID
0,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
1,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
2,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
3,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
4,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001
...,...,...,...
56,2023-12-14T17:38:00-CME-001,2023-12-14 17:38:00+00:00,2023-12-18T06:00:00-GST-001
57,2024-02-28T17:48:00-CME-001,2024-02-28 17:48:00+00:00,2024-03-03T18:00:00-GST-001
58,2024-03-23T01:25:00-CME-001,2024-03-23 01:25:00+00:00,2024-03-24T12:00:00-GST-001
59,2024-03-23T01:48:00-CME-001,2024-03-23 01:48:00+00:00,2024-03-24T12:00:00-GST-001


### GST Data

In [15]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
gst_url = base_url+GST+"?startDate="+startDate+"&endDate="+endDate+"&api_key="+NASA_API_KEY

In [16]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(gst_url)

In [17]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json[0],indent=4))

{
    "gstID": "2013-06-01T01:00:00-GST-001",
    "startTime": "2013-06-01T01:00Z",
    "allKpIndex": [
        {
            "observedTime": "2013-06-01T01:00Z",
            "kpIndex": 6.0,
            "source": "NOAA"
        }
    ],
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
    "linkedEvents": [
        {
            "activityID": "2013-05-31T15:45:00-HSS-001"
        }
    ],
    "submissionTime": "2013-07-15T19:26Z",
    "versionId": 1
}


In [18]:
# Convert gst_json to a Pandas DataFrame  
gst_df = pd.DataFrame(gst_json)
# Keep only the columns: activityID, startTime, linkedEvents
gst_df = gst_df[["gstID","startTime","linkedEvents"]]
gst_df

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,None
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
...,...,...,...
112,2023-12-18T06:00:00-GST-001,2023-12-18T06:00Z,[{'activityID': '2023-12-14T17:38:00-CME-001'}...
113,2024-03-03T18:00:00-GST-001,2024-03-03T18:00Z,[{'activityID': '2024-02-28T17:48:00-CME-001'}...
114,2024-03-23T21:00:00-GST-001,2024-03-23T21:00Z,[{'activityID': '2024-03-23T12:49:00-IPS-001'}...
115,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,[{'activityID': '2024-03-23T01:25:00-CME-001'}...


In [19]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df = gst_df.dropna(subset="linkedEvents").reset_index(drop=True)
gst_df

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
3,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
4,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...
...,...,...,...
100,2023-12-18T06:00:00-GST-001,2023-12-18T06:00Z,[{'activityID': '2023-12-14T17:38:00-CME-001'}...
101,2024-03-03T18:00:00-GST-001,2024-03-03T18:00Z,[{'activityID': '2024-02-28T17:48:00-CME-001'}...
102,2024-03-23T21:00:00-GST-001,2024-03-23T21:00Z,[{'activityID': '2024-03-23T12:49:00-IPS-001'}...
103,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,[{'activityID': '2024-03-23T01:25:00-CME-001'}...


In [20]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df_expanded = gst_df.explode("linkedEvents",ignore_index=True).dropna(subset="linkedEvents").reset_index(drop=True)
gst_df_expanded

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'}
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'}
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'}
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'}
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'}
...,...,...,...
200,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-23T01:48:00-CME-001'}
201,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-24T14:10:00-IPS-001'}
202,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-24T16:25:00-MPC-001'}
203,2024-04-19T18:00:00-GST-001,2024-04-19T18:00Z,{'activityID': '2024-04-15T06:48:00-CME-001'}


In [21]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
gst_df_expanded["CME_ActivityID"] = gst_df_expanded["linkedEvents"].apply(lambda x: extract_activityID_from_dict(x))
# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_df_expanded = gst_df_expanded.dropna(subset="CME_ActivityID").reset_index(drop=True)
gst_df_expanded

,gstID,startTime,linkedEvents,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'},2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'},2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'},2013-10-02T02:47:00-MPC-001
...,...,...,...,...
200,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-23T01:48:00-CME-001'},2024-03-23T01:48:00-CME-001
201,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-24T14:10:00-IPS-001'},2024-03-24T14:10:00-IPS-001
202,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,{'activityID': '2024-03-24T16:25:00-MPC-001'},2024-03-24T16:25:00-MPC-001
203,2024-04-19T18:00:00-GST-001,2024-04-19T18:00Z,{'activityID': '2024-04-15T06:48:00-CME-001'},2024-04-15T06:48:00-CME-001


In [22]:
# Convert the 'CME_ActivityID' column to string format 
gst_df_expanded["CME_ActivityID"] = gst_df_expanded["CME_ActivityID"].astype("string")
# Convert the 'gstID' column to string format 
gst_df_expanded["gstID"] = gst_df_expanded["gstID"].astype("string")
# Convert startTime to datetime format  
gst_df_expanded["startTime"] = pd.to_datetime(gst_df_expanded["startTime"])
# Rename startTime to startTime_GST 
gst_df_expanded.rename(columns={"startTime":"startTime_GST"},inplace=True)
# Drop linkedEvents
gst_df_expanded.drop(columns=["linkedEvents"],inplace=True)
# Verify that all steps were executed correctly
gst_df_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           205 non-null    string             
 1   startTime_GST   205 non-null    datetime64[ns, UTC]
 2   CME_ActivityID  205 non-null    string             
dtypes: datetime64[ns, UTC](1), string(2)
memory usage: 4.9 KB


In [23]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
gst_df_expanded = gst_df_expanded[gst_df_expanded["CME_ActivityID"].str.contains("CME")].reset_index(drop=True)
gst_df_expanded

,gstID,startTime_GST,CME_ActivityID
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001
...,...,...,...
56,2023-12-18T06:00:00-GST-001,2023-12-18 06:00:00+00:00,2023-12-14T17:38:00-CME-001
57,2024-03-03T18:00:00-GST-001,2024-03-03 18:00:00+00:00,2024-02-28T17:48:00-CME-001
58,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:25:00-CME-001
59,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:48:00-CME-001


### Merge both datatsets

In [49]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
data_joined = cme_df_expanded.merge(gst_df_expanded,left_on=["GST_ActivityID","cmeID"],right_on=["gstID", "CME_ActivityID"])
data_joined

,cmeID,startTime_CME,GST_ActivityID,gstID,startTime_GST,CME_ActivityID
0,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
1,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
2,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
3,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
4,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001
...,...,...,...,...,...,...
56,2023-12-14T17:38:00-CME-001,2023-12-14 17:38:00+00:00,2023-12-18T06:00:00-GST-001,2023-12-18T06:00:00-GST-001,2023-12-18 06:00:00+00:00,2023-12-14T17:38:00-CME-001
57,2024-02-28T17:48:00-CME-001,2024-02-28 17:48:00+00:00,2024-03-03T18:00:00-GST-001,2024-03-03T18:00:00-GST-001,2024-03-03 18:00:00+00:00,2024-02-28T17:48:00-CME-001
58,2024-03-23T01:25:00-CME-001,2024-03-23 01:25:00+00:00,2024-03-24T12:00:00-GST-001,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:25:00-CME-001
59,2024-03-23T01:48:00-CME-001,2024-03-23 01:48:00+00:00,2024-03-24T12:00:00-GST-001,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:48:00-CME-001


In [50]:
# Verify that the new DataFrame has the same number of rows as cme and gst
print(f"cme_df_expanded row count: {cme_df_expanded.shape[0]}")
print(f"gst_df_expanded row count: {gst_df_expanded.shape[0]}")
print(f"data_joined     row count: {data_joined.shape[0]}")

cme_df_expanded row count: 61
gst_df_expanded row count: 61
data_joined     row count: 61


### Computing the time it takes for a CME to cause a GST

In [51]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
data_joined["timeDiff"] = abs(data_joined["startTime_GST"] - data_joined["startTime_CME"])
data_joined

,cmeID,startTime_CME,GST_ActivityID,gstID,startTime_GST,CME_ActivityID,timeDiff
0,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,4 days 06:36:00
1,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,2 days 04:20:00
2,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,3 days 00:48:00
3,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,2 days 12:45:00
4,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,2 days 01:35:00
...,...,...,...,...,...,...,...
56,2023-12-14T17:38:00-CME-001,2023-12-14 17:38:00+00:00,2023-12-18T06:00:00-GST-001,2023-12-18T06:00:00-GST-001,2023-12-18 06:00:00+00:00,2023-12-14T17:38:00-CME-001,3 days 12:22:00
57,2024-02-28T17:48:00-CME-001,2024-02-28 17:48:00+00:00,2024-03-03T18:00:00-GST-001,2024-03-03T18:00:00-GST-001,2024-03-03 18:00:00+00:00,2024-02-28T17:48:00-CME-001,4 days 00:12:00
58,2024-03-23T01:25:00-CME-001,2024-03-23 01:25:00+00:00,2024-03-24T12:00:00-GST-001,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:25:00-CME-001,1 days 10:35:00
59,2024-03-23T01:48:00-CME-001,2024-03-23 01:48:00+00:00,2024-03-24T12:00:00-GST-001,2024-03-24T12:00:00-GST-001,2024-03-24 12:00:00+00:00,2024-03-23T01:48:00-CME-001,1 days 10:12:00


In [52]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
data_joined.describe().loc[["mean","50%"]]

,timeDiff
mean,2 days 23:29:26.557377049
50%,2 days 19:51:00


### Exporting data in csv format

In [53]:
# Export data to CSV without the index
data_joined.to_csv("./output/outputfile.csv",index=False)